# Idenfication of nearest ORFs

In [22]:
import os
import sys
import pandas as pd
print(sys.version)

3.11.3 (main, May 15 2023, 10:43:03) [Clang 14.0.6 ]


# Identify additional ORFs adjacent to the nearest ORF for each peak
### The search is limited to 1000 bp each direction of the peak 

In [15]:
py_script = "~/GitProjects/geisingerlab/geisingerlab-chipseq-bfmrs/src/nearest_ORF_additional.py"
infile = "~/2023_ChipSeq/average_peak_summit/BfmR-ChIP-49_seed1.average_summit.bed"
outfile = "~/2023_ChipSeq/peak_stat_next_ORFs/BfmR-ChIP-49_seed1.nextORFs.tsv"
annotations = "~/2023_ChipSeq/annotations"

In [16]:
cmd = ' '.join(['python', py_script, infile, outfile, annotations])
os.system(cmd)

0

In [17]:
!wc -l $outfile

     332 /Users/yunfei/2023_ChipSeq/peak_stat_next_ORFs/BfmR-ChIP-49_seed1.nextORFs.tsv


# Filter matched ORFs by activated / repressed call from RNAseq table

In [20]:
rnaseq_bfmRS = "~/2023_ChipSeq/rnaseq/bfmRS_WT.xlsx"
rnaseq_bfmR = "~/2023_ChipSeq/rnaseq/bfmR_WT.xlsx"
rnaseq_bfmS = "~/2023_ChipSeq/rnaseq/bfmS_WT.xlsx"

ORF_table = "~/2023_ChipSeq/peak_stat_next_ORFs/BfmR-ChIP-49_seed1.nextORFs.tsv"
ORF_RNAseq = "~/2023_ChipSeq/peak_stat_next_ORFs/BfmR-ChIP-49_seed1.nextORFs.RNASeq.tsv"


In [21]:
def read_rnaseq_table(file):
    df = pd.read_excel(file)
    df = df.set_index("gene_id")
    return df

In [ ]:
rnaseq_table = {}
rnaseq_table.update({"BfmRS_WT" : read_rnaseq_table(rnaseq_bfmRS)})
rnaseq_table.update({"BfmR_WT" : read_rnaseq_table(rnaseq_bfmR)})
rnaseq_table.update({"BfmS_WT" : read_rnaseq_table(rnaseq_bfmS)})

df_chipseq = pd.read_csv(ORF_table, sep='\t', index_col=0)
df_chipseq = df_chipseq.set_index("locus_tag")

col_names = ["log2(fold_change)", "significant", "protein_id", "product"]

for i,j in rnaseq_table.items():
    df_rnaseq = j
    for row, column in df_chipseq.iterrows():
        for k in col_names:
            new_column_name = i + "_" + k
            try:
                df_chipseq.loc[row, new_column_name] = df_rnaseq.loc[row,k]
            except:
                pass
df_chipseq.to_csv(ORF_RNAseq, sep='\t')